# ClimateHack.AI 2023: Training a Basic Model

Thank you for participating in ClimateHack.AI 2023! 🌍

Your contributions could help cut carbon emissions by up to 100 kilotonnes per year in Great Britain alone. We look forward to seeing what you build over the course of the competition!

In this Jupyter notebook, you will hopefully train your first model for the challenge using historical solar PV data and HRV satellite imagery.

For more detailed information on the challenge, see the [DOXA AI competition page](https://doxaai.com/competition/climatehackai-2023/overview). 😎

## Installing packages

Before you can get started, you will need to install a number of packages to allow you to work with the data and submit to the platform. If you do not already have these packages installed, you can uncomment the lines below to do so! You will also need to [install PyTorch](https://pytorch.org/get-started/locally/).

In [24]:
%pip install  xarray
# %pip install -U doxa-cli 

Note: you may need to restart the kernel to use updated packages.


DEPRECATION: pyitlib 0.2.2 has a non-standard dependency specifier pandas>=0.20.2numpy>=1.9.2. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyitlib or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


## Importing packages

Here, we import a number of packages we will need to train our first model.

In [ ]:
import json
import os
import uuid
from typing import Any, Dict, List, Optional
import torch
import xarray
from google.cloud import aiplatform, storage
from google_cloud_pipeline_components.preview.automl.forecasting import \
    utils as automl_forecasting_utils

In [ ]:
PROJECT_ID = "solar-pv-yeilds"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}
REGION = "us-central1" 
# ! gcloud auth login
aiplatform.init(project=PROJECT_ID, location=REGION)

In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}


device(type='cuda')

## Creating your submission directory

If you cloned [this repository](https://github.com/climatehackai/getting-started-2023) straight from GitHub, you will already have all the files you need, but if you are running this notebook using Google Colab, we just need to download a couple extra files to create a fresh submission directory that you will soon hopefully be in a position to upload to the platform as part of your first competition submission.


In [ ]:
if not os.path.exists("submission"):
    os.makedirs("submission", exist_ok=True)

    !curl -L https://raw.githubusercontent.com/climatehackai/getting-started-2023/main/submission/competition.py --output submission/competition.py
    !curl -L https://raw.githubusercontent.com/climatehackai/getting-started-2023/main/submission/doxa.yaml --output submission/doxa.yaml
    !curl -L https://raw.githubusercontent.com/climatehackai/getting-started-2023/main/submission/model.py --output submission/model.py
    !curl -L https://raw.githubusercontent.com/climatehackai/getting-started-2023/main/submission/run.py --output submission/run.py
    !curl -L https://raw.githubusercontent.com/climatehackai/getting-started-2023/main/indices.json --output indices.json

## Downloading a month of data

While streaming the Zarr-format datasets directly from Hugging Face was adequate for some initial data exploration in `1_data.ipynb`, it most likely will not be fast enough in training. Since there is so much data available, we can get started just by downloading a single month of PV and HRV satellite imagery data.

In [13]:
if not os.path.exists("data"):
    os.makedirs("data/pv/2020", exist_ok=True)
    os.makedirs("data/satellite-hrv/2020", exist_ok=True)

    !curl -L https://huggingface.co/datasets/climatehackai/climatehackai-2023/resolve/main/pv/metadata.csv --output data/pv/metadata.csv
    !curl -L https://huggingface.co/datasets/climatehackai/climatehackai-2023/resolve/main/pv/2020/7.parquet --output data/pv/2020/7.parquet
    !curl -L https://huggingface.co/datasets/climatehackai/climatehackai-2023/resolve/main/satellite-hrv/2020/7.zarr.zip --output data/satellite-hrv/2020/7.zarr.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
 14 1300k   14  192k    0     0   877k      0  0:00:01 --:--:--  0:00:01  884k
100 1300k  100 1300k    0     0  4212k      0 --:--:-- --:--:-- --:--:-- 4235k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  1122  100  1122    0     0  11699      0 --:--:-- --:--:-- --:--:-- 12329

  7 74.3M    7 5976k    0     0  7279k      0  0:00:10 --:--:--  0:00:10 7279k
 29 74.3M   29 21.7M    0     0  11.9M      0  0:00:06  0:00:01  0:00:05 15.9M
 48 74.3M   48 36.1M    0     0  9763k      0  0:00:07  0:00:03  0:00:04 10.2M
 48 74.3M   48 36.1M    0     0  8513k      0  0:

## Loading the data

In [14]:
pv = pd.read_parquet("data/pv/2020/7.parquet").drop("generation_wh", axis=1)

pv

power
timestamp                 ss_id       
2020-07-01 00:00:00+00:00 2607     0.0
                          2626     0.0
                          2631     0.0
                          2657     0.0
                          2729     0.0
...                                ...
2020-07-31 23:55:00+00:00 27062    0.0
                          27063    0.0
                          27064    0.0
                          27065    0.0
                          27066    0.0

[7827134 rows x 1 columns]

In [25]:
hrv = xr.open_dataset(
    "data/satellite-hrv/2020/7.zarr.zip", engine="zarr", chunks="auto"
)

hrv

ValueError: unrecognized engine zarr must be one of: ['scipy', 'store']

As part of the challenge, you can make use of satellite imagery, numerical weather prediction and air quality forecast data in a `[128, 128]` region centred on each solar PV site. In order to help you out, we have pre-computed the indices corresponding to each solar PV site and included them in `indices.json`, which we can load directly. For more information, take a look at the [challenge page](https://doxaai.com/competition/climatehackai-2023).


In [ ]:
with open("indices.json") as f:
    site_locations = {
        data_source: {
            int(site): (int(location[0]), int(location[1]))
            for site, location in locations.items()
        }
        for data_source, locations in json.load(f).items()
    }

### Defining a PyTorch Dataset

To get started, we will define a simple `IterableDataset` that shows how to slice into the PV and HRV data using `pandas` and `xarray`, respectively. You will have to modify this if you wish to incorporate non-HRV data, weather forecasts and air quality forecasts into your training regimen. If you have any questions, feel free to ask on the [ClimateHack.AI Community Discord server](https://discord.gg/HTTQ8AFjJp)!

**Note**: `site_locations` contains indices for the non-HRV, weather forecast and air quality forecast data as well as for the HRV data!

There are many more advanced strategies you could implement to load data in training, particularly if you want to pre-prepare training batches in advance or use multiple workers to improve data loading times.

In [ ]:
class ChallengeDataset(IterableDataset):
    def __init__(self, pv, hrv, site_locations, sites=None):
        self.pv = pv
        self.hrv = hrv
        self._site_locations = site_locations
        self._sites = sites if sites else list(site_locations["hrv"].keys())

    def _get_image_times(self):
        min_date = datetime(2020, 7, 1)
        max_date = datetime(2020, 7, 30)

        start_time = time(8)
        end_time = time(17)

        date = min_date
        while date <= max_date:
            current_time = datetime.combine(date, start_time)
            while current_time.time() < end_time:
                if current_time:
                    yield current_time

                current_time += timedelta(minutes=60)

            date += timedelta(days=1)

    def __iter__(self):
        for time in self._get_image_times():
            first_hour = slice(str(time), str(time + timedelta(minutes=55)))

            pv_features = pv.xs(first_hour, drop_level=False)  # type: ignore
            pv_targets = pv.xs(
                slice(  # type: ignore
                    str(time + timedelta(hours=1)),
                    str(time + timedelta(hours=4, minutes=55)),
                ),
                drop_level=False,
            )

            hrv_data = self.hrv["data"].sel(time=first_hour).to_numpy()

            for site in self._sites:
                try:
                    # Get solar PV features and targets
                    site_features = pv_features.xs(site, level=1).to_numpy().squeeze(-1)
                    site_targets = pv_targets.xs(site, level=1).to_numpy().squeeze(-1)
                    assert site_features.shape == (12,) and site_targets.shape == (48,)

                    # Get a 128x128 HRV crop centred on the site over the previous hour
                    x, y = self._site_locations["hrv"][site]
                    hrv_features = hrv_data[:, y - 64 : y + 64, x - 64 : x + 64, 0]
                    assert hrv_features.shape == (12, 128, 128)

                    # How might you adapt this for the non-HRV, weather and aerosol data?
                except:
                    continue

                yield site_features, hrv_features, site_targets

## Defining a model

In order to make a PyTorch-based submission to the DOXA AI platform, you need to upload both the code defining your model in addition to your trained model weights (and some code to run your model). As a result, if you want to experiment with different model architectures using this notebook, you will need to edit the model in `submission/model.py` and re-import it here.

Here is the small convolutional neural network you are initially given in `submission/model.py`. You will absolutely be able to improve upon this!

```py
#########################################
#       Improve this basic model!       #
#########################################

class Model(torch.nn.Module):
    def __init__(self) -> None:
        super().__init__()

        self.conv1 = nn.Conv2d(in_channels=12, out_channels=24, kernel_size=3)
        self.conv2 = nn.Conv2d(in_channels=24, out_channels=48, kernel_size=3)
        self.conv3 = nn.Conv2d(in_channels=48, out_channels=96, kernel_size=3)
        self.conv4 = nn.Conv2d(in_channels=96, out_channels=192, kernel_size=3)

        self.pool = nn.MaxPool2d(kernel_size=2)
        self.flatten = nn.Flatten()

        self.linear1 = nn.Linear(6924, 48)

    def forward(self, pv, hrv):
        x = torch.relu(self.pool(self.conv1(hrv)))
        x = torch.relu(self.pool(self.conv2(x)))
        x = torch.relu(self.pool(self.conv3(x)))
        x = torch.relu(self.pool(self.conv4(x)))

        x = self.flatten(x)
        x = torch.concat((x, pv), dim=-1)

        x = torch.sigmoid(self.linear1(x))

        return x

```

In [ ]:
# Import the model defined in `submission/model.py`

from submission.model import Model

In [ ]:
summary(Model(), input_size=[(1, 12), (1, 12, 128, 128)])

## Train a model

In [ ]:
BATCH_SIZE = 32

dataset = ChallengeDataset(pv, hrv, site_locations=site_locations)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, pin_memory=True)

In [ ]:
model = Model().to(device)

criterion = nn.L1Loss()
optimiser = optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
EPOCHS = 1

for epoch in range(EPOCHS):
    model.train()

    running_loss = 0.0
    count = 0
    for i, (pv_features, hrv_features, pv_targets) in enumerate(dataloader):
        optimiser.zero_grad()

        predictions = model(
            pv_features.to(device, dtype=torch.float),
            hrv_features.to(device, dtype=torch.float),
        )

        loss = criterion(predictions, pv_targets.to(device, dtype=torch.float))
        loss.backward()

        optimiser.step()

        size = int(pv_targets.size(0))
        running_loss += float(loss) * size
        count += size

        if i % 200 == 199:
            print(f"Epoch {epoch + 1}, {i + 1}: {running_loss / count}")

    print(f"Epoch {epoch + 1}: {running_loss / count}")

In [ ]:
# Save your model
torch.save(model.state_dict(), "submission/model.pt")

# Submitting to the DOXA AI platform

Congratulations &ndash; **you have trained your first model for ClimateHack.AI 2023**! 🥳

Why not try making a submission to the platform?

First, make sure you have enrolled for the competition on the [ClimateHack.AI 2023 competition page](https://doxaai.com/competition/climatehackai-2023). You will need to be signed in with a DOXA AI account registered with your university email address so that we can verify your eligibility.

You can then sign in with the CLI using the following command:

In [ ]:
!doxa login

Finally, you can upload your submission to the platform by running the following cell:

In [ ]:
!doxa upload submission

If everything went well, you will soon appear on the [competition scoreboard](https://doxaai.com/competition/climatehackai-2023/scoreboard) once your model has been evaluated! 😎

## Next steps

Well done for reaching the end of this Jupyter notebook! By now, you will have loaded and explored the data, trained a basic model, and joined other competition participants on the [competition scoreboard](https://doxaai.com/competition/climatehackai-2023/scoreboard)!

To get started, we used a very simple model architecture, but this model most likely does not have a sufficiently rich representation to properly solve our problem. How might you be able to improve on this? Which model architectures would be best suited to this problem? Would you want to train a model from scratch, as we have done here, or possibly fine-tune a pre-trained computer vision model? Check out the resources on the [competition page](https://doxaai.com/competition/climatehackai-2023) for ideas on where to go from here.

Additionally, we only used historical PV and HRV data, but perhaps you might be able to get more mileage out of the other data sources available to you, such as non-HRV satellite imagery, the DWD weather forecast data or even the aerosol data. If you do decide to incorporate more data, what **data engineering** work would you have to perform so that you can train effectively on a large quantity of data?

**We want to hear about your approaches**! If you develop anything interesting, let us know on the [ClimateHack.AI Community Discord server](https://discord.gg/HTTQ8AFjJp) and start a conversation!